# Project 2 - Ames Housing Data and Kaggle Challenge - Cleaning
## Matt Reed / DSI-124


In [401]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [402]:
def dummy_corr(column_name, dataframe, target_name):
    return dataframe[column_name].str.get_dummies().join(dataframe[target_name]).corr()[target_name].sort_values(ascending=False)[1:]

In [403]:
# Approach found at https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
def flatten(t):
    return [item for sublist in t for item in sublist]

In [404]:
df_train = pd.read_csv('../datasets/train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [405]:
# MS SubClass uses numbers for labels; should not be treated as str
df_train['MS SubClass'] = df_train['MS SubClass'].astype(str)

In [406]:
# Looking at null values
df_train.isnull().sum().sort_values(ascending=False)

Pool QC         2042
Misc Feature    1986
Alley           1911
Fence           1651
Fireplace Qu    1000
                ... 
Heating QC         0
Central Air        0
Electrical         0
1st Flr SF         0
SalePrice          0
Length: 81, dtype: int64

In [407]:
# Realized that Year Sold was obfuscated by being treated as an integer
df_train['Yr Sold'] = df_train['Yr Sold'].astype(str)
df_train['Mo Sold'] = df_train['Mo Sold'].astype(str)

In [408]:
df_train.select_dtypes(include=['object']).describe().transpose()

,count,unique,top,freq
MS SubClass,2051,16,20,770
MS Zoning,2051,7,RL,1598
Street,2051,2,Pave,2044
Alley,140,2,Grvl,85
Lot Shape,2051,4,Reg,1295
Land Contour,2051,4,Lvl,1843
Utilities,2051,3,AllPub,2049
Lot Config,2051,5,Inside,1503
Land Slope,2051,3,Gtl,1953
Neighborhood,2051,28,NAmes,310


In [409]:
df_train['Yr Sold'].describe()

count     2051
unique       5
top       2007
freq       498
Name: Yr Sold, dtype: object

In [410]:
dummy_corr('MS SubClass', df_train, 'SalePrice').max()

0.3554214620632621

In [411]:
df_train.select_dtypes(include=['object']).columns

Index(['MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Lot Shape',
       'Land Contour', 'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood',
       'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond',
       'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating',
       'Heating QC', 'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature',
       'Mo Sold', 'Yr Sold', 'Sale Type'],
      dtype='object')

In [412]:
# Creating a list of the highest correlations based on dummifying categorical variables in my dataframe
categorical_corr = [(column_name, 
                     dummy_corr(column_name, df_train, 'SalePrice').max(), 
                     dummy_corr(column_name, df_train, 'SalePrice').min(), 
                     dummy_corr(column_name, df_train, 'SalePrice').max() - dummy_corr(column_name, df_train, 'SalePrice').min()) 
 for column_name 
 in df_train.select_dtypes(include=['object']).columns]

In [413]:
# Sort approach found on https://pythonguides.com/python-sort-list-of-tuples/
cat_corr_diff = categorical_corr.copy()

cat_corr_diff.sort(reverse=True, key= lambda x: x[3])
cat_corr_diff[:]

[('Exter Qual', 0.49386101668262256, -0.6003620438785177, 1.0942230605611403),
 ('Kitchen Qual', 0.5512844949973896, -0.540860057110634, 1.0921445521080235),
 ('Bsmt Qual', 0.586497229460346, -0.45697898944878185, 1.0434762189091278),
 ('Foundation', 0.5290468529844157, -0.35553521520011966, 0.8845820681845353),
 ('Garage Finish',
  0.4229363608708119,
  -0.43222009547586315,
  0.855156456346675),
 ('Heating QC', 0.4532553529286872, -0.3435415579170743, 0.7967969108457615),
 ('Mas Vnr Type', 0.31002603603789, -0.4240875482859905, 0.7341135843238804),
 ('Garage Type', 0.35787902815727923, -0.3703444386576489, 0.7282234668149281),
 ('Bsmt Exposure',
  0.3770317587162424,
  -0.2931059611409868,
  0.6701377198572291),
 ('Neighborhood', 0.4486468134029907, -0.2083710127541118, 0.6570178261571025),
 ('BsmtFin Type 1',
  0.4635488990371277,
  -0.15091467927234825,
  0.614463578309476),
 ('MS SubClass', 0.3554214620632621, -0.24853416697324687, 0.6039556290365089),
 ('Lot Shape', 0.27357444442

In [414]:
categorical_corr.copy().sort(reverse=True, key= lambda x: x[2])

In [415]:
df_train.nunique().sort_values()[:20]

Central Air       2
Street            2
Alley             2
Paved Drive       3
Half Bath         3
Garage Finish     3
Utilities         3
Bsmt Half Bath    3
Land Slope        3
Bsmt Full Bath    4
Mas Vnr Type      4
Kitchen AbvGr     4
Exter Qual        4
Land Contour      4
Lot Shape         4
Pool QC           4
Bsmt Exposure     4
Fence             4
Kitchen Qual      4
Heating           5
dtype: int64

In [416]:
corr_price = df_train.corr()['SalePrice']
corr_price_sorted = corr_price[:-1].sort_values(ascending=False)
corr_price_sorted

Overall Qual       0.800207
Gr Liv Area        0.697038
Garage Area        0.650270
Garage Cars        0.648220
Total Bsmt SF      0.628925
1st Flr SF         0.618486
Year Built         0.571849
Year Remod/Add     0.550370
Full Bath          0.537969
Garage Yr Blt      0.533922
Mas Vnr Area       0.512230
TotRms AbvGrd      0.504014
Fireplaces         0.471093
BsmtFin SF 1       0.423519
Lot Frontage       0.341842
Open Porch SF      0.333476
Wood Deck SF       0.326490
Lot Area           0.296566
Bsmt Full Bath     0.283662
Half Bath          0.283001
2nd Flr SF         0.248452
Bsmt Unf SF        0.190210
Bedroom AbvGr      0.137067
Screen Porch       0.134581
3Ssn Porch         0.048732
Pool Area          0.023106
BsmtFin SF 2       0.016255
Misc Val          -0.007375
Low Qual Fin SF   -0.041594
Bsmt Half Bath    -0.045328
Id                -0.051398
Overall Cond      -0.097019
Kitchen AbvGr     -0.125444
Enclosed Porch    -0.135656
PID               -0.255052
Name: SalePrice, dty

Features of Interest:

In [417]:
corr_top = list(corr_price_sorted[corr_price_sorted > .5].index.values)
corr_top

['Overall Qual',
 'Gr Liv Area',
 'Garage Area',
 'Garage Cars',
 'Total Bsmt SF',
 '1st Flr SF',
 'Year Built',
 'Year Remod/Add',
 'Full Bath',
 'Garage Yr Blt',
 'Mas Vnr Area',
 'TotRms AbvGrd']

In [418]:
cat_corr_top = [variable for variable, high, low, diff in categorical_corr if diff > .5]
cat_corr_top

['MS SubClass',
 'MS Zoning',
 'Lot Shape',
 'Neighborhood',
 'Roof Style',
 'Exterior 1st',
 'Exterior 2nd',
 'Mas Vnr Type',
 'Exter Qual',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'Heating QC',
 'Central Air',
 'Kitchen Qual',
 'Garage Type',
 'Garage Finish',
 'Paved Drive',
 'Sale Type']

In [419]:
list_of_lists = [['Id'], corr_top, cat_corr_top, ['Mo Sold', 'Yr Sold'], ['SalePrice']]
columns_of_interest = flatten(list_of_lists)

In [420]:
columns_of_interest

['Id',
 'Overall Qual',
 'Gr Liv Area',
 'Garage Area',
 'Garage Cars',
 'Total Bsmt SF',
 '1st Flr SF',
 'Year Built',
 'Year Remod/Add',
 'Full Bath',
 'Garage Yr Blt',
 'Mas Vnr Area',
 'TotRms AbvGrd',
 'MS SubClass',
 'MS Zoning',
 'Lot Shape',
 'Neighborhood',
 'Roof Style',
 'Exterior 1st',
 'Exterior 2nd',
 'Mas Vnr Type',
 'Exter Qual',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'Heating QC',
 'Central Air',
 'Kitchen Qual',
 'Garage Type',
 'Garage Finish',
 'Paved Drive',
 'Sale Type',
 'Mo Sold',
 'Yr Sold',
 'SalePrice']

In [421]:
# Pair down dataframe to columns of interest
df_reduced = df_train[columns_of_interest]

In [422]:
df_test = pd.read_csv('../datasets/test.csv')
df_test_reduced = df_test[columns_of_interest[:-1]]

In [423]:
df_reduced.columns

Index(['Id', 'Overall Qual', 'Gr Liv Area', 'Garage Area', 'Garage Cars',
       'Total Bsmt SF', '1st Flr SF', 'Year Built', 'Year Remod/Add',
       'Full Bath', 'Garage Yr Blt', 'Mas Vnr Area', 'TotRms AbvGrd',
       'MS SubClass', 'MS Zoning', 'Lot Shape', 'Neighborhood', 'Roof Style',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Foundation', 'Bsmt Qual', 'Bsmt Exposure', 'BsmtFin Type 1',
       'Heating QC', 'Central Air', 'Kitchen Qual', 'Garage Type',
       'Garage Finish', 'Paved Drive', 'Sale Type', 'Mo Sold', 'Yr Sold',
       'SalePrice'],
      dtype='object')

In [425]:
df_test_reduced.columns

Index(['Id', 'Overall Qual', 'Gr Liv Area', 'Garage Area', 'Garage Cars',
       'Total Bsmt SF', '1st Flr SF', 'Year Built', 'Year Remod/Add',
       'Full Bath', 'Garage Yr Blt', 'Mas Vnr Area', 'TotRms AbvGrd',
       'MS SubClass', 'MS Zoning', 'Lot Shape', 'Neighborhood', 'Roof Style',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Foundation', 'Bsmt Qual', 'Bsmt Exposure', 'BsmtFin Type 1',
       'Heating QC', 'Central Air', 'Kitchen Qual', 'Garage Type',
       'Garage Finish', 'Paved Drive', 'Sale Type', 'Mo Sold', 'Yr Sold'],
      dtype='object')

In [426]:
# Want to include Month and Year Sold
modified_cat_corr_top = cat_corr_top.copy()
modified_cat_corr_top.append('Mo Sold')
modified_cat_corr_top

['MS SubClass',
 'MS Zoning',
 'Lot Shape',
 'Neighborhood',
 'Roof Style',
 'Exterior 1st',
 'Exterior 2nd',
 'Mas Vnr Type',
 'Exter Qual',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'Heating QC',
 'Central Air',
 'Kitchen Qual',
 'Garage Type',
 'Garage Finish',
 'Paved Drive',
 'Sale Type',
 'Mo Sold']

In [427]:
# Dummify categorical variables
df_reduced = pd.get_dummies(df_reduced, columns=modified_cat_corr_top, drop_first=True)

df_reduced

,Id,Overall Qual,Gr Liv Area,Garage Area,Garage Cars,Total Bsmt SF,1st Flr SF,Year Built,Year Remod/Add,Full Bath,...,Mo Sold_11,Mo Sold_12,Mo Sold_2,Mo Sold_3,Mo Sold_4,Mo Sold_5,Mo Sold_6,Mo Sold_7,Mo Sold_8,Mo Sold_9
0,109,6,1479,475.0,2.0,725.0,725,1976,2005,2,...,0,0,0,1,0,0,0,0,0,0
1,544,7,2122,559.0,2.0,913.0,913,1996,1997,2,...,0,0,0,0,1,0,0,0,0,0
2,153,5,1057,246.0,1.0,1057.0,1057,1953,2007,1,...,0,0,0,0,0,0,0,0,0,0
3,318,5,1444,400.0,2.0,384.0,744,2006,2007,2,...,0,0,0,0,1,0,0,0,0,0
4,255,6,1445,484.0,2.0,676.0,831,1900,1993,2,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,8,1728,520.0,2.0,1884.0,1728,2007,2007,2,...,0,0,0,0,0,0,0,0,0,0
2047,785,4,861,539.0,2.0,861.0,861,1940,1950,1,...,0,0,0,1,0,0,0,0,0,0
2048,916,6,1913,342.0,2.0,896.0,1172,1928,1950,1,...,0,0,0,1,0,0,0,0,0,0
2049,639,4,1200,294.0,1.0,1200.0,1200,1956,1956,1,...,1,0,0,0,0,0,0,0,0,0


In [428]:
# Dummify test dataframe identically to main dataframe
df_test_reduced = pd.get_dummies(df_test_reduced, columns=cat_corr_top, drop_first=True)

In [429]:
# Approach found at https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data

# Get missing columns in the training test
missing_cols = set(df_reduced.columns) - set(df_test_reduced.columns)
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    df_test_reduced[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
df_test_reduced = df_test_reduced[df_reduced.columns]

In [430]:
df_test_reduced

,Id,Overall Qual,Gr Liv Area,Garage Area,Garage Cars,Total Bsmt SF,1st Flr SF,Year Built,Year Remod/Add,Full Bath,...,Mo Sold_11,Mo Sold_12,Mo Sold_2,Mo Sold_3,Mo Sold_4,Mo Sold_5,Mo Sold_6,Mo Sold_7,Mo Sold_8,Mo Sold_9
0,2658,6,1928,440,1,1020,908,1910,1950,2,...,0,0,0,0,0,0,0,0,0,0
1,2718,5,1967,580,2,1967,1967,1977,1977,2,...,0,0,0,0,0,0,0,0,0,0
2,2414,7,1496,426,2,654,664,2006,2006,2,...,0,0,0,0,0,0,0,0,0,0
3,1989,5,968,480,2,968,968,1923,2006,1,...,0,0,0,0,0,0,0,0,0,0
4,625,6,1394,514,2,1394,1394,1963,1963,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,6,1877,488,2,1084,1084,1974,1974,2,...,0,0,0,0,0,0,0,0,0,0
874,1234,6,1988,480,2,1104,1104,1966,1999,2,...,0,0,0,0,0,0,0,0,0,0
875,1373,5,1211,322,1,952,1211,1968,1968,1,...,0,0,0,0,0,0,0,0,0,0
876,1672,4,864,528,2,864,864,1971,1971,1,...,0,0,0,0,0,0,0,0,0,0


In [431]:
# Relatively small number of null values
df_reduced.isnull().sum().sum()/len(df_reduced.index)

0.06777181862506095

In [432]:
df_reduced.dropna(inplace=True)

In [433]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1915 entries, 0 to 2050
Columns: 158 entries, Id to Mo Sold_9
dtypes: float64(5), int64(9), object(1), uint8(143)
memory usage: 506.8+ KB


In [434]:
df_reduced.to_csv('../datasets/filtered_dataset.csv', index=False)

In [435]:
df_test_reduced.to_csv('../datasets/filtered_test_dataset.csv', index=False)